### Practical Lab 3 - Vanilla CNN and Fine-Tune VGG16 - for Dogs and Cats Classification

Cemil Caglar Yapici - 9081058

This notebook demonstrates a complete workflow for classifying images of dogs vs cats using TensorFlow/Keras. We first load and explore the data, then train two models – a simple CNN from scratch and a fine-tuned VGG16 – and finally evaluate their performance.

NOTE!!: I worked on 1250 cat +1250 dog photos, otherwise It crashed my pc and Too big for to upload on Github.

### 1. Obtain the Data

We assume the dataset is organized in folders Dog vs Cat/train/ and Dog vs Cat/test/. The training folder contains labeled images (cat.*.jpg and dog.*.jpg), while the test folder contains unlabeled images. We will split a portion of the training set for validation.

In [ ]:
import os
import numpy as np
import random
import tensorflow as tf

# Set up GPU usage (fall back to CPU if GPU unavailable)
gpu_name = tf.test.gpu_device_name()
if gpu_name:
    print("GPU detected:", gpu_name)
else:
    print("No GPU found, using CPU")

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Paths to data directories
data_dir = 'Dog vs Cat'
train_dir = os.path.join(data_dir, 'train')
test_dir  = os.path.join(data_dir, 'test')

# Verify counts of dog and cat images in the training set
train_files = [f for f in os.listdir(train_dir) if f.endswith('.jpg')]
num_dogs = sum(f.startswith('dog') for f in train_files)
num_cats = sum(f.startswith('cat') for f in train_files)
print(f"Total training images: {len(train_files)} (Dogs: {num_dogs}, Cats: {num_cats})")


In [ ]:
# Prepare file paths and labels
file_paths = [os.path.join(train_dir, fname) for fname in train_files]
labels = [1 if fname.startswith('dog') else 0 for fname in train_files]

# Shuffle and split the data
indices = np.arange(len(file_paths))
np.random.shuffle(indices)
file_paths = np.array(file_paths)[indices]
labels = np.array(labels)[indices]
split_idx = int(0.8 * len(file_paths))
train_paths, train_labels = file_paths[:split_idx], labels[:split_idx]
val_paths,   val_labels   = file_paths[split_idx:], labels[split_idx:]

print(f"Training samples: {len(train_paths)}, Validation samples: {len(val_paths)}")


### 2. Exploratory Data Analysis (EDA)

### 2.1 Display Sample Images
We visualize a few random training images with their labels to ensure data looks correct.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Show a few random training images with labels
plt.figure(figsize=(10,4))
for i in range(6):
    idx = np.random.randint(len(train_paths))
    img = mpimg.imread(train_paths[idx])
    plt.subplot(2, 3, i+1)
    plt.imshow(img)
    plt.title("Dog" if train_labels[idx]==1 else "Cat")
    plt.axis('off')
plt.suptitle("Random Dog and Cat Images from Training Set")
plt.show()


### 2.2 Class Distribution
Count how many images per class to check for imbalance.

In [ ]:
import seaborn as sns

sns.barplot(x=['Cat', 'Dog'], y=[num_cats, num_dogs], palette=['#4c72b0','#c44e52'])
plt.ylabel('Count')
plt.title('Class Distribution in Training Set')
plt.show()


### 2.3 Image Statistics
Compute basic statistics on image sizes and shapes.

In [ ]:
from PIL import Image

widths = []; heights = []
for path in train_paths:
    img = Image.open(path)
    w, h = img.size
    widths.append(w); heights.append(h)

print(f"Image width: min={min(widths)}, max={max(widths)}, avg={np.mean(widths):.1f}")
print(f"Image height: min={min(heights)}, max={max(heights)}, avg={np.mean(heights):.1f}")


### 3. Model Training

We train two models:

a) Simple CNN from scratch.

b) VGG16 with Transfer Learning (pretrained on ImageNet) + fine-tuning.

### 3.1 Data Pipeline

First, we create a TensorFlow input pipeline for training and validation. We will use two versions: one that scales pixels to [0,1] for the simple CNN, and one that applies VGG16 preprocessing (mean subtraction) for the VGG model. We resize all images to 224×224 pixels.

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def preprocess_simple(file_path, label):
    # Load and resize image, scale pixel values to [0,1]
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0
    return img, label

def preprocess_vgg(file_path, label):
    # Load and resize image, apply VGG16 preprocessing
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.keras.applications.vgg16.preprocess_input(img)  # subtracts mean RGB values
    return img, label

# Create training and validation datasets for simple CNN
train_ds_simple = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds_simple = (train_ds_simple
                   .map(preprocess_simple, num_parallel_calls=tf.data.AUTOTUNE)
                   .shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

val_ds_simple = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds_simple = (val_ds_simple
                 .map(preprocess_simple, num_parallel_calls=tf.data.AUTOTUNE)
                 .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

# Create training and validation datasets for VGG (same split)
train_ds_vgg = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds_vgg = (train_ds_vgg
                .map(preprocess_vgg, num_parallel_calls=tf.data.AUTOTUNE)
                .shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

val_ds_vgg = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds_vgg = (val_ds_vgg
              .map(preprocess_vgg, num_parallel_calls=tf.data.AUTOTUNE)
              .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))


### 3.2 Simple CNN

We build a straightforward convolutional neural network with a few Conv2D and MaxPooling layers, followed by a dense classifier. This is a binary classifier with a sigmoid output. Simpler CNNs have been shown to perform well on Cats vs Dogs classification

In [ ]:
from tensorflow.keras import layers, models

model_cnn = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(*IMG_SIZE, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model_cnn.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

model_cnn.summary()


In [ ]:
callbacks_cnn = [
    tf.keras.callbacks.ModelCheckpoint('best_simple_cnn.h5', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
]

history_cnn = model_cnn.fit(
    train_ds_simple,
    epochs=15,
    validation_data=val_ds_simple,
    callbacks=callbacks_cnn
)


### 3.3 Fine-Tuned VGG16

We now use transfer learning with Keras’ pretrained VGG16 (trained on ImageNet). First, we freeze all layers of VGG16 and add a new classifier head. Then we optionally unfreeze some top layers to fine-tune
tensorflow.org
. This approach often yields high accuracy even on small datasets

In [ ]:
from tensorflow.keras.applications import VGG16

# Load VGG16 base model without top layers, with imagenet weights
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(*IMG_SIZE,3))
base_model.trainable = False  # freeze base

# Build new top layers for our binary task
vgg_input = base_model.input
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)
model_vgg = models.Model(inputs=vgg_input, outputs=output)

model_vgg.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy', metrics=['accuracy'])
model_vgg.summary()


In [ ]:
callbacks_vgg = [
    tf.keras.callbacks.ModelCheckpoint('best_vgg.h5', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
]

history_vgg = model_vgg.fit(
    train_ds_vgg,
    epochs=10,
    validation_data=val_ds_vgg,
    callbacks=callbacks_vgg
)


Fine-tuning: After initial training, we unfreeze some top layers of VGG16 for further training, allowing the pretrained features to adapt to our data
tensorflow.org
. Here we unfreeze the last convolutional block.

In [ ]:
# Unfreeze last convolutional block of VGG16
for layer in base_model.layers[-4:]:
    layer.trainable = True

model_vgg.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss='binary_crossentropy', metrics=['accuracy'])

history_vgg_fine = model_vgg.fit(
    train_ds_vgg,
    epochs=5,
    validation_data=val_ds_vgg,
    callbacks=callbacks_vgg
)


### 3.4 Training Curves:
Plot training/validation accuracy and loss for both models to check learning and overfitting.

In [ ]:
import matplotlib.pyplot as plt

# Simple CNN training curves
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history_cnn.history['accuracy'], label='Train Acc')
plt.plot(history_cnn.history['val_accuracy'], label='Val Acc')
plt.title('Simple CNN Accuracy')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
plt.subplot(1,2,2)
plt.plot(history_cnn.history['loss'], label='Train Loss')
plt.plot(history_cnn.history['val_loss'], label='Val Loss')
plt.title('Simple CNN Loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
plt.show()

# VGG training curves
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history_vgg.history['accuracy'] + history_vgg_fine.history['accuracy'], label='Train Acc')
plt.plot(history_vgg.history['val_accuracy'] + history_vgg_fine.history['val_accuracy'], label='Val Acc')
plt.title('VGG16 Accuracy')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
plt.subplot(1,2,2)
plt.plot(history_vgg.history['loss'] + history_vgg_fine.history['loss'], label='Train Loss')
plt.plot(history_vgg.history['val_loss'] + history_vgg_fine.history['val_loss'], label='Val Loss')
plt.title('VGG16 Loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
plt.show()


### 4 Model Evaluation

We load the best saved weights and evaluate both models on the held-out validation set. We compute accuracy, confusion matrix, precision, recall, F1-score, and plot precision-recall curves. We also show some misclassified images.

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve

# Load best weights
model_cnn.load_weights('best_simple_cnn.h5')
model_vgg.load_weights('best_vgg.h5')

# Utility to evaluate model
def evaluate_model(model, dataset, true_labels):
    # Get predictions
    probs = model.predict(dataset)
    preds = (probs > 0.5).astype(int).flatten()
    acc = np.mean(preds == true_labels)
    cm = confusion_matrix(true_labels, preds)
    prec = precision_score(true_labels, preds)
    rec = recall_score(true_labels, preds)
    f1 = f1_score(true_labels, preds)
    # Precision-Recall curve data
    prec_vals, rec_vals, _ = precision_recall_curve(true_labels, probs)
    return acc, cm, prec, rec, f1, (prec_vals, rec_vals)

# Prepare validation data in NumPy arrays for metrics
val_images_simple = []
val_images_vgg = []
for img, lbl in val_ds_simple.unbatch():
    val_images_simple.append(img.numpy())
for img, lbl in val_ds_vgg.unbatch():
    val_images_vgg.append(img.numpy())

val_images_simple = np.array(val_images_simple)
val_images_vgg = np.array(val_images_vgg)
y_true = val_labels  # same labels

# Evaluate Simple CNN
acc_cnn, cm_cnn, prec_cnn, rec_cnn, f1_cnn, (p_cnn, r_cnn) = evaluate_model(model_cnn, val_ds_simple, y_true)
print(f"Simple CNN -- Accuracy: {acc_cnn:.3f}, Precision: {prec_cnn:.3f}, Recall: {rec_cnn:.3f}, F1: {f1_cnn:.3f}")
print("Confusion Matrix (rows=true, cols=predicted):\n", cm_cnn)

# Evaluate VGG
acc_vgg, cm_vgg, prec_vgg, rec_vgg, f1_vgg, (p_vgg, r_vgg) = evaluate_model(model_vgg, val_ds_vgg, y_true)
print(f"VGG16 -- Accuracy: {acc_vgg:.3f}, Precision: {prec_vgg:.3f}, Recall: {rec_vgg:.3f}, F1: {f1_vgg:.3f}")
print("Confusion Matrix (rows=true, cols=predicted):\n", cm_vgg)


### 4.1 Precision-Recall Curves

Plot precision vs recall at different thresholds for each model

In [ ]:
plt.figure(figsize=(6,5))
plt.plot(r_cnn, p_cnn, label='Simple CNN')
plt.plot(r_vgg, p_vgg, label='VGG16')
plt.xlabel('Recall'); plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(); plt.grid(True); plt.show()


### 4.2 Misclassified Examples
Display some images that were misclassified by each model (true vs predicted label).

In [ ]:
import random
def show_misclassified(model, dataset, true_labels, title):
    # Gather all predictions
    probs = model.predict(dataset).flatten()
    preds = (probs > 0.5).astype(int)
    mis_idx = np.where(preds != true_labels)[0]
    # Show up to 6 misclassified samples
    plt.figure(figsize=(10,4))
    for i, idx in enumerate(random.sample(list(mis_idx), min(6, len(mis_idx)))):
        img = mpimg.imread(val_paths[idx])
        plt.subplot(2, 3, i+1)
        plt.imshow(img)
        plt.title(f"True: {'Dog' if true_labels[idx]==1 else 'Cat'}\nPred: {'Dog' if preds[idx]==1 else 'Cat'}")
        plt.axis('off')
    plt.suptitle(f"Misclassified by {title}")
    plt.show()

show_misclassified(model_cnn, val_ds_simple, y_true, title="Simple CNN")
show_misclassified(model_vgg, val_ds_vgg, y_true, title="VGG16")


### 5. Conclusions

Basic Convolutional Neural Network: 

- This basic model can train on data comparatively faster and can achieve decent accuracy levels. As mentioned in other studies and in our case too, this basic CNN model achieved around 80 to 85 % validation accuracy on Cats vs Dogs. This has also been represented in its confusion matrix and F1 score.

VGG16 Transfer Learning: 

- Using VGG16 that has been trained on ImageNet before and fine tuning it with further training usually results in attaining a better accuracy, especially on small data. Transfer learning improves small data generalization because it utilizes data from a larger dataset. In our results, VGG16 had better precision and recall Compared to the basic CNN. 

Overfitting and Regularization: 

- Having training and validation plots makes it easier to identify Overfitting. We implemented dropout and early stopping. If too many layers are unfrozen, the fine tuned VGG16 can overfit the small dataset. 

Metric Evaluation: 

- In order to identify the results, we used accuracy, confusion matrix, precision, recall and F1. Recall (TPR) tells us how many of the actual dogs (or cats) we managed to find correctly, while precision tells us how many of the predicted dogs are actually dogs.

Advice: 

- The best model for practical use would be to use either the Fine-Tuned VGG16 or some other new model. This model would give you both accuracy and strength; however, if you have limited computation power, use a most likely a well-tuned basic convolutional neural network (CNN). A lot of time simple convolutional networks can work just as well for this binary problem stated by a certain research article. Ultimately it will depend on your priorities, whether you want faster processing speeds or to get highly accurate results.